# VGG-16 Work

In [30]:
import timm

model = timm.create_model("hf_hub:timm/vgg16.tv_in1k", pretrained=True)

ModuleNotFoundError: No module named 'timm'

### Importing Libraries

In [ ]:
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint # type: ignore
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

### Read the Data

In [ ]:
# Import the utility functions
from animal_classification import get_data_generators, display_translated_labels

# Set the directory path for images
directory = r'csv/raw-img'

# Get the data generators
train_generator, validation_generator = get_data_generators(directory)

# Display and translate class labels
class_labels, translated_labels = display_translated_labels(train_generator)

### Building VGG Model

In [ ]:
# Define the input dimensions
img_height, img_width = 224, 224 

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [22]:
# Add custom layers on top of the VGG16 base
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector per image
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
predictions = Dense(10, activation='softmax')(x)  # Final output layer for 10 classes

model = Model(inputs=base_model.input, outputs=predictions)

In [26]:
model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=["accuracy", "Precision", "Recall"]
)

In [28]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6),
    ModelCheckpoint('vgg16.keras', save_best_only=True)
]

In [29]:
# Training
history = model.fit(train_generator,
    validation_data=validation_generator,
    epochs=5,
    callbacks=callbacks)

Epoch 1/5
696/696 [==============================] - 2380s 3s/step - loss: 0.9691 - accuracy: 0.6788 - precision: 0.8641 - recall: 0.5020 - val_loss: 0.6817 - val_accuracy: 0.7820 - val_precision: 0.8836 - val_recall: 0.6813 - lr: 0.0010
Epoch 2/5
696/696 [==============================] - 2628s 4s/step - loss: 0.6534 - accuracy: 0.7802 - precision: 0.8694 - recall: 0.6996 - val_loss: 0.6171 - val_accuracy: 0.7907 - val_precision: 0.8773 - val_recall: 0.7254 - lr: 0.0010
Epoch 3/5
213/696 [========>.....................] - ETA: 27:26 - loss: 0.5827 - accuracy: 0.8072 - precision: 0.8730 - recall: 0.7406

KeyboardInterrupt: 

In [ ]:
def plot_training_history(history, metrics, title="Training and Validation Metrics", save_path=None):
    valid_metrics = [metric for metric in metrics if metric in history.history]
    num_metrics = len(valid_metrics)
    rows = (num_metrics + 1) // 2
    fig, axs = plt.subplots(rows, 2, figsize=(14, 5 * rows))
    axs = axs.flatten()

    fig.suptitle(title, fontsize=16)

    for ax, metric in zip(axs, valid_metrics):
        ax.plot(history.history[metric], label=f'Training {metric.capitalize()}', color='blue', linestyle='-', marker='o')
        ax.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}', color='orange', linestyle='--', marker='x')
        ax.set_xlabel('Epochs')
        ax.set_ylabel(metric.capitalize())
        ax.set_title(f'Training vs. Validation {metric.capitalize()}')
        ax.legend()
        ax.grid(True)
        ax.set_ylim(0, 1)

    for i in range(num_metrics, len(axs)):
        fig.delaxes(axs[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    if save_path:
        plt.savefig(save_path)
    
    plt.show()

metrics = ['accuracy', 'loss', 'recall', 'precision']

In [ ]:
plot_training_history(history, metrics)

In [ ]:
y_test = validation_generator.classes
y_pred = np.argmax(model.predict(validation_generator), axis=1)
print(classification_report(y_test, y_pred, target_names=translated_labels))

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=translated_labels, yticklabels=translated_labels)
plt.show()